In [19]:
#!pip3 install pandas
#!pip3 install numpy
#!pip3 install sklearn
#!pip3 install category_encoders
!pip3 install xgboost

     |████████████████████████████████| 1.7 MB 8.7 MB/s eta 0:00:01


In [3]:
import pandas as pd
import numpy as np

In [4]:
raw_csv = pd.read_csv('/Users/yunheekim/codestates/Section3/project3/heart_900.csv')

In [5]:
raw_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [6]:
raw_csv.shape

(918, 12)

In [10]:
raw_csv.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [17]:
import sklearn

from sklearn.model_selection import train_test_split

train, test = train_test_split(raw_csv, train_size= 0.8, test_size=0.2)

print(train.shape, test.shape)

## train, val, test 가 아닌 train, test 로만 분리해서 진행

(734, 12) (184, 12)


In [12]:
## train 데이터 셋 타겟 분포

train['HeartDisease'].value_counts()

1    414
0    320
Name: HeartDisease, dtype: int64

In [15]:
## Train / Test data set - Features 와 Target 분리

def divide_dataset(df):

    target = 'HeartDisease'

    features = df.drop(columns = [target]).columns

    X = df[features]
    y = df[target]

    return X, y

X_train, y_train = divide_dataset(train)
X_test, y_test = divide_dataset(test)

X_train.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
688,76,F,NAP,140,197,0,ST,116,N,1.1,Flat
712,58,M,ASY,100,234,0,Normal,156,N,0.1,Up
827,43,F,NAP,122,213,0,Normal,165,N,0.2,Flat
58,54,M,ASY,150,365,0,ST,134,N,1.0,Up
845,61,F,ASY,145,307,0,LVH,146,Y,1.0,Flat


In [20]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier

import xgboost
from xgboost import XGBClassifier

In [26]:
## 모델링

def fit_model(X_train, y_train):

    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(),
        XGBClassifier()
    )

    dists = {
        'simpleimputer__strategy' : ['mean', 'median', 'most_frequent'],
        'xgbclassifier__n_estimator' : range(1, 500),
        'xgbclassifier__max_depth' : range(3, 20),
        'xgbclassifier__learning_rate' : [0.1, 0.2, 0.3],
        'xgbclassifier__min_child_weight' : range(0, 30)
    }

    clf = RandomizedSearchCV(
        pipe,
        param_distributions = dists,
        n_iter = 10,
        cv = 5,
        scoring = 'f1',
        n_jobs = -1
    )

    clf.fit(X_train, y_train)

    return clf

clf = fit_model(X_train, y_train)
print("Best CV F1 :", clf.best_score_)
print("Best CV params :", clf.best_params_)

[19:00:02] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:00:02] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:00:02] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_6

In [27]:
y_test_pred = clf.best_estimator_.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print('Test accuracy :', test_accuracy)
print('Test F1 :', test_f1)

Test accuracy : 0.8586956521739131
Test F1 : 0.8659793814432989


In [ ]:
## 모델링 85% 로 완료